Convert the text to vecor and built classification (to categorys) model

 Import librarys

In [131]:
import pandas as pd
import gensim
import numpy as np
import io
import requests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

 Import data

In [132]:
url = "https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
# df = pd.read_csv('bbc-text.csv')  # use this line instead if you've downloaded the dataset directly

First look

In [133]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [134]:
print('Number of news items: {}'.format(len(df)))

Number of news items: 2225


Change data format (the X)

In [135]:
def newFormat(df):
    newFormat = []
    for doc in df['text']:
        newFormat.append(gensim.utils.simple_preprocess(doc))
    return newFormat

In [136]:
documents = newFormat(df)

Initialize W2V model

In [137]:
model = gensim.models.Word2Vec(documents, size=100, window=10, min_count=2, workers=10)

Train model on CBOW fake task

In [138]:
model.train(
    sentences=documents,
    total_examples=len(documents),
    epochs=10,
)

(6435895, 8222420)

Get the vector- representation for one word

In [139]:
model.wv.get_vector("terror")

array([-0.247948  ,  1.4817705 ,  1.1731193 , -0.7352575 , -3.6172955 ,
       -0.70589536, -0.44925013,  1.2104946 , -0.05808148, -2.0127046 ,
       -0.8022793 , -0.53043973,  1.8797314 , -0.8468813 ,  1.086041  ,
        3.1321883 ,  0.04044692, -0.40190345, -1.0103875 , -0.12973264,
        0.874239  , -2.0074866 ,  1.6917236 ,  0.44042015,  1.4375659 ,
        0.4889775 ,  0.44908187, -1.1053171 , -0.43586943, -0.09784798,
        0.16780566,  1.4072573 ,  0.32650167, -0.01123121,  0.739822  ,
        1.2174685 ,  1.1417632 ,  0.32388577,  0.11824897, -2.11777   ,
        0.1680798 ,  1.4567606 ,  0.68481225,  0.05593596, -1.9518185 ,
        1.1506207 ,  0.18451706,  0.53328824,  0.690894  , -1.9359596 ,
       -0.06833078,  0.34403017,  2.6562395 ,  1.5126411 ,  0.60132253,
        1.7620566 ,  0.17247364, -0.18452024,  1.2712307 ,  0.07717162,
       -1.8230402 , -0.8769561 ,  0.11878794, -0.2641311 ,  1.0668143 ,
       -2.0956762 ,  0.29802695, -1.4310032 , -0.14438011,  0.17

Get the 8 most similar words to a specific word 

In [140]:
model.wv.most_similar(positive="terror", topn=8)

[('terrorism', 0.8959832787513733),
 ('suspects', 0.848685085773468),
 ('laws', 0.8424480557441711),
 ('detention', 0.8422465324401855),
 ('arrest', 0.8240644335746765),
 ('troops', 0.7899489402770996),
 ('indefinitely', 0.7893084287643433),
 ('terrorist', 0.7793425917625427)]

Get the 6 most similar words to severl words

In [141]:
model.wv.most_similar(positive=["france", "germany"], topn=6)

[('sweden', 0.7754604816436768),
 ('australia', 0.7615177631378174),
 ('argentina', 0.7568036913871765),
 ('italy', 0.7437316179275513),
 ('israel', 0.7342903017997742),
 ('switzerland', 0.725965142250061)]

Odd one out

In [142]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["france","germany","sudan"])

C:\Users\user\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'sudan'

similarity = Distance -> Cosine similarity = Euclidean distance

In [143]:
# similarity between two identical words
model.wv.similarity(w1="dirty", w2="dirty")

1.0

Convert the documents with the W2V model

In [144]:
def document_to_vector(document, model):
    token_list = gensim.utils.simple_preprocess(document)
    vector_list = []
    for token in token_list:
        try:
            vector_list.append(model.wv[token])
        except KeyError:
            pass
    return np.average(vector_list, axis=0)

In [145]:
df['text'].iloc[0] #documents[0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [146]:
vec0 = document_to_vector(df['text'].iloc[0], model)
print(vec0.shape)

(100,)


In [147]:
print(vec0)

[-0.3701799   0.13209751  0.22073975 -0.23600435  0.8523257  -0.22078095
 -0.37087724  0.47804114  0.36813343  0.03797998 -0.06981204 -0.29123738
  0.02499889  0.56150997  0.24686511  0.09720914  0.36006552 -0.7107542
 -0.14941545  0.21751909  0.2797646  -0.2624107   0.4171261  -0.11035814
  0.891785    0.31406993 -0.4740122   0.51410604  0.14935772  0.0245052
 -0.08344772  0.07206858  0.4135765  -0.07684539  0.65363854 -0.35316613
 -0.2900219  -0.07408021 -0.0686966  -0.21096973  0.18807237  0.04141556
 -0.63421446 -0.16423182  0.37279463 -0.09958774  0.27124074 -0.10471367
  0.29457396 -0.06792044 -0.20915729 -0.07134391  0.46079785  0.5831285
  0.5233121   0.2897043   0.14836197  0.27877876  0.10949034 -1.0096508
  0.42499343  0.06301104 -0.35908246  0.13453579  0.11513744 -0.5041391
  0.30018607  0.69333     0.61220944  0.18549065  0.23318413 -0.27428105
  0.0312994   0.9538004   0.17198525  0.280109   -0.18974283  0.7744005
  0.07087917 -0.08866408  0.60607266 -0.28381485 -0.13259

In [148]:
df['w2v'] = df['text'].apply(document_to_vector, model=model)

In [149]:
df.head()

,category,text,w2v
0,tech,tv future in the hands of viewers with home th...,"[-0.3701799, 0.13209751, 0.22073975, -0.236004..."
1,business,worldcom boss left books alone former worldc...,"[0.089996286, 0.53134274, 0.15382855, -0.12444..."
2,sport,tigers wary of farrell gamble leicester say ...,"[-0.049190007, 0.6095951, 0.6979966, -0.040894..."
3,sport,yeading face newcastle in fa cup premiership s...,"[0.65634936, 0.75915796, 0.8789271, 0.02282835..."
4,entertainment,ocean s twelve raids box office ocean s twelve...,"[0.5947537, 0.24263561, 0.030684646, -0.421718..."


Encode category (y)

In [150]:
lblenc = LabelEncoder()
df['lbl'] = lblenc.fit_transform(df['category'])

In [151]:
lblenc.classes_

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [152]:
df.head()

,category,text,w2v,lbl
0,tech,tv future in the hands of viewers with home th...,"[-0.3701799, 0.13209751, 0.22073975, -0.236004...",4
1,business,worldcom boss left books alone former worldc...,"[0.089996286, 0.53134274, 0.15382855, -0.12444...",0
2,sport,tigers wary of farrell gamble leicester say ...,"[-0.049190007, 0.6095951, 0.6979966, -0.040894...",3
3,sport,yeading face newcastle in fa cup premiership s...,"[0.65634936, 0.75915796, 0.8789271, 0.02282835...",3
4,entertainment,ocean s twelve raids box office ocean s twelve...,"[0.5947537, 0.24263561, 0.030684646, -0.421718...",1


Split data before model fitting

In [153]:
X = np.stack(df['w2v'].values)
y = df['lbl'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Train model

In [154]:
rndforest = RandomForestClassifier(n_estimators=50, max_depth=8)
rndforest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Test model

In [155]:
y_pred = rndforest.predict(X_test)
print(classification_report(y_test, y_pred, target_names=lblenc.classes_))

               precision    recall  f1-score   support

     business       0.97      0.93      0.95       181
entertainment       0.94      0.93      0.93       129
     politics       0.90      0.97      0.93       125
        sport       1.00      0.97      0.99       158
         tech       0.96      0.98      0.97       142

     accuracy                           0.96       735
    macro avg       0.95      0.96      0.95       735
 weighted avg       0.96      0.96      0.96       735

